In [22]:
from config import TWITTER_DATA_DIR
from sklearn.decomposition import PCA
import os, hdbscan, numpy as np, pickle, time, pandas as pd
import preprocessing

## Load Data and Reduce Dimensionality

In [6]:
with open(os.path.join(TWITTER_DATA_DIR, 'cascade_root_embeddings.pkl'), 'rb') as f:
    corpus_embeddings = pickle.load(f)

X_sbert = np.vstack(corpus_embeddings).T
print('shape:', X_sbert.shape)

shape: (768, 767082)


In [7]:
pca_transform = PCA(n_components=70)
transformed_data = pca_transform.fit_transform(X_sbert.T)

In [9]:
var = pca_transform.explained_variance_ratio_.cumsum()[69]
print('explained variance:', var)

explained variance: 0.8796812662161906


## Clustering

In [ ]:
algorithm = hdbscan.HDBSCAN
algorithm_kwargs = dict(min_cluster_size=2, allow_single_cluster=False, metric='manhattan',
                       core_dist_n_jobs=4, cluster_selection_epsilon=25)

clusterer = algorithm(**algorithm_kwargs)
start_time = time.time()
labels = clusterer.fit_predict(transformed_data)

with open(os.path.join(TWITTER_DATA_DIR, 'clustering_labels_epsilon_25_pca_70.pkl'), 'wb') as f:
    pickle.dump(labels, f)

print('done, time elapsed:', time.time() - start_time)

## Exploring Clusters

In [15]:
del corpus_embeddings # too large

In [13]:
with open(os.path.join(TWITTER_DATA_DIR, 'cascade_root_ids.pkl'), 'rb') as f:
    cascade_root_ids = pickle.load(f) 

In [14]:
cascade_root_ids[0]

'1081428026049601536'

In [18]:
campaigns = preprocessing.load_campaign()
all_cascades = preprocessing.process_campaign(campaigns)

100%|██████████| 75/75 [00:26<00:00,  2.84it/s]


In [19]:
root_to_cascade = {c.root.id: c for c in all_cascades}

In [31]:
df = pd.DataFrame(data=labels, index=cascade_root_ids, columns=['Cluster'])

In [35]:
# add othet attributes
df['text'] = [root_to_cascade[x].root.text for x in df.index]
df['username'] = [root_to_cascade[x].root.username for x in df.index]
df['template'] = [root_to_cascade[x].root.template for x in df.index]

In [88]:
df['date'] = [root_to_cascade[x].root.date for x in df.index]

In [89]:
sizes = df.groupby('Cluster').count().text.rename('Cluster_Size')

In [90]:
df['cluster_size'] = df.Cluster.apply(lambda x: sizes.get(x))

In [81]:
df[df.text.str.contains('292.74')]

,Cluster,text,username,template,cluster_size
1097017559705104384,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi",sunilsingh_bjp,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70
1097035537041375232,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi",kunal829122,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70
1097055315005067265,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू , गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा । #jharkhandwithmodi 14",bjp4simdega,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70
1097055840949874689,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू , गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा । #jharkhandwithmodi",bjp4chatra,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70
1097059011524845570,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi",reachsapna007,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70
1097059046681600002,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi",jharsocial,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70
1097059563767967744,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi",kajalnishad13,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70
1097059704662970369,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू , गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा । #jharkhandwithmodi",bjp4jharkhand,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70
1097059597318184962,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi",vandana63956451,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70
1097059891338989568,17640,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू , गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा । #jharkhandwithmodi bjp4jharkhand",bjp4jamtara,"करीब 292.74 करोड़ की लागत से बने पलामू मेडिकल कॉलेज से पलामू, गढ़वा और लातेहार ज़िलों के 55 लाख से ज्यादा लोगों को फायदा होगा। #jharkhandwithmodi\n",70


In [101]:
df.query("template == '' and Cluster != -1 and cluster_size < 10 and cluster_size > 5").tail(70)

,Cluster,text,username,template,cluster_size,date
1050342327187857409,19271,"rt nehakak5 ""so air force chief, french government, dassault, french president, supreme court are all lying and what rahul gandhi, whos government did one scam after another during 2004-2014 is the only one speaking truth? what a big big joke! #gappupappu nsitharraman""",khakrafafda,,6,2018-10-11 11:08:19
1050342324327391232,9417,"rt nehakak5 ""rahul gandhi is totally frustrated now because he couldn’t do a rafale scam under upa and benefit robert vadra and his aides. dassault papers do not say anywhere that reliance was mandatory. #gappupappu nsitharraman""",khakrafafda,,6,2018-10-11 11:08:19
1050342329918398465,7718,"rt nehakak5 ""annual defence ministerial dialogue between india-france was decided way back in march 2018 between pm modi and macron. this visit by nsitharaman did not happen immediately after yesterday. rahul gandhi should know facts before telling lies. #gappupappu nsitha…",khakrafafda,,9,2018-10-11 11:08:20
1050342331927461888,6042,"rt nehakak5 "".rahulgandhi and company is touching new low every day to sabotage rafale deal. latest example is his lie about rm's france visit while nsitharaman was already scheduled to visit paris. #gappupappu""",khakrafafda,,9,2018-10-11 11:08:21
1050342543219720193,7893,rt khandelw13sagar: this is ridiculous. nowhere does the document accessed by mediapart mentions reliance was necessary. it only mentions that indian offset was mandatory. incindia pidis are peddling lies again on their master’s orders. #gappupappu,bk_chudasama,,7,2018-10-11 11:09:00
...,...,...,...,...,...,...
1051082311926730757,5109,"@mynareshpurohit @yesimsaffron_ लोग विश्वास दिलाने के वास्ते कहते हैं- ""मालिक मैंने आपका नमक खाया है"" पर कॉंग्रेसी कहते हैं- ""मालिक हमने आपका मूत पिया है"" 😂😂 #pappumutra #gappupappu #modifor2019 #metoo",sharmaritesh777,,6,2018-10-13 12:08:45
1051082598213144576,5109,"लोग विश्वास दिलाने के वास्ते कहते हैं- ""मालिक मैंने आपका नमक खाया है"" पर कॉंग्रेसी कहते हैं- ""मालिक हमने आपका मूत पिया है"" 😂😂 #pappumutra #gappupappu #modifor2019 #metoo",sharmaritesh777,,6,2018-10-13 12:09:54
1051154760231936002,5109,"लोग विश्वास दिलाने के वास्ते कहते हैं- मालिक मैंने आपका नमक खाया है"" पर कॉंग्रेसी कहते हैं- ""मालिक हमने आपका मूत पिया है"" #pappumutra #gappupappu #modifor2019 #metoo pic.twitter.com/qvjuq6nfdy""",anubabag,,6,2018-10-13 16:56:00
1051155389884985344,5109,"लोग विश्वास दिलाने के वास्ते कहते हैं- मालिक मैंने आपका नमक खाया है"" पर कॉंग्रेसी कहते हैं- ""मालिक हमने आपका मूत पिया है"" #pappumutra #gappupappu #modifor2019 #metoo pic.twitter.com/k42pgyitgh""",anubabag,,6,2018-10-13 16:59:00


In [44]:
pd.set_option('display.max_colwidth', -1) 

<ipython-input-44-b7cc634e6674>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [102]:
neighborhood = set([5109+x for x in range(-3,3)])
df[df.Cluster.isin(neighborhood)].sort_values('Cluster')

,Cluster,text,username,template,cluster_size,date
1098469106620088321,5106,around 84% prefer @narendramodi ji as prime minister if general elections were held today as per survey #timesmegapoll by country's leading media group @timesofindia #लक्ष्य_हमारा_मोदी_दोबारा #2019elections https://t.co/z9cg1xkz3g,patidarmanasvi,,2,2019-02-21 06:26:58
1098848340836413441,5106,around 84% prefer @narendramodi ji as prime minister if general elections were held today as per survey #timesmegapoll by country's leading media group @timesofindia #लक्ष्य_हमारा_मोदी_दोबारा #2019elections https://t.co/mlwhgmmfhz,jatbarad,,2,2019-02-22 07:33:54
1110723647415771138,5107,50 % increase in railways track renewal #nris4modi,devkarlekar,50% increase in railways track renewal #nris4modi\n,8,2019-03-27 02:02:00
1109784914939920385,5107,उत्तर प्रदेश में दो साल कार्यकाल में ट्रैक नवीनीकरण में 50% वृद्धि #उत्तरप्रदेश_विजय_संकल्प https://t.co/3h8b3don4n https://t.co/efbyejfvyb,saurabh79462159,,8,2019-03-24 11:51:57
1110718627832057857,5107,50% increase in railways track renewal #nris4modi,superma06749121,50% increase in railways track renewal #nris4modi\n,8,2019-03-27 01:42:11
1110719114299871232,5107,50% increase in railways track renewal #nris4modi,ddebtech,50% increase in railways track renewal #nris4modi\n,8,2019-03-27 01:44:07
1110725743921848320,5107,50% increase in railways track renewal #nris4modi https://t.co/rs13zxnjq5,jayasreenair1,50% increase in railways track renewal #nris4modi\n,8,2019-03-27 02:10:28
1110727058361995265,5107,50% increase in railways track renewal #nris4modi,amadne09,50% increase in railways track renewal #nris4modi\n,8,2019-03-27 02:15:41
1110728925917908992,5107,50% increase in railways track renewal #nris4modi,oonattu,50% increase in railways track renewal #nris4modi\n,8,2019-03-27 02:23:06
1110730328753299459,5107,50% increase in railways track renewal #nris4modi .@narendramodi,digambere,50% increase in railways track renewal #nris4modi\n,8,2019-03-27 02:28:41


In [91]:
df.query('Cluster == 223')

,Cluster,text,username,template,cluster_size,date
1092291241763459072,223,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra,mancikati,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra\n,8,2019-02-04 05:18:20
1092301579674279937,223,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra,alapativijaybh1,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra\n,8,2019-02-04 05:59:25
1092304687196487680,223,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra,ajayvar18747201,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra\n,8,2019-02-04 06:11:46
1092306188899835904,223,the polavaram irrigation project is completely funded by the centre . #bjp4betterandhra,bjp4atp,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra\n,8,2019-02-04 06:17:00
1092307859822456832,223,lie : modi govt. doing nothing for polavaram project fact :the polavaram irrigation project is completely funded by the centre. so far 7000 cr released #bjp4betterandhra,yoursdharm,,8,2019-02-04 06:24:22
1092314935961370624,223,the polavaram irrigation project is completely funded by the centre . #bjp4betterandhra,adikya2,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra\n,8,2019-02-04 06:52:00
1092315416234463232,223,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra,neerajr81070454,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra\n,8,2019-02-04 06:54:24
1092371498424102912,223,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra,raghuvamsi85,the polavaram irrigation project is completely funded by the centre. #bjp4betterandhra\n,8,2019-02-04 10:37:15


In [38]:
df.sort_values('Cluster', ascending=False).head(20)

,Cluster,text,username,template
1124220147781103616,45583,great initiative by @pmoindia @narendramodi ji...,bsc10001,
1106650379960438795,45583,amazing!! @rahulgandhi @priyankagandhi @digvij...,sonurohit,
1108449733356355584,45583,awesome 👏👏👊👊👊#mainbhichowkidar #phirekbaarmodi...,dev3321,
1106948382772228101,45583,@narendramodi .@avinashchoubey amazing work br...,prohilla535,
1108737130854010882,45583,awesome it is 👍 @narendramodi @bjp4india #main...,bjpishaan,
1108015348404510723,45583,😂😂😂 another brilliant one @narendramodi @bjp4i...,save_d_nation,
1106948382772228101,45583,@narendramodi .@avinashchoubey amazing work br...,prohilla535,
1091250722719420417,45583,अद्भुत। #लक्ष्य_हमारा_मोदी_दोबारा #namoagain @...,atulghenghat,
1101487043861872641,45582,#bjpvijaysankalpbikerally,shobhi_baraiya,
1101487045543747584,45582,#bjpvijaysankalpbikerally,shobhi_baraiya,
